# 2. Closed-loop simulation and visualization

Please note that [RigidBodySim.jl](https://github.com/JuliaRobotics/RigidBodySim.jl) now provides a more capable simulation environment.

## Setup

In [1]:
using Pkg # hide
Pkg.activate("/home/travis/build/JuliaRobotics/RigidBodyDynamics.jl/docs/../examples/2. Closed-loop simulation and visualization") # hide
Pkg.instantiate() # hide
using RigidBodyDynamics

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Installed IniFile ─────────────────── v0.5.0
 Installed Ratios ──────────────────── v0.3.1
 Installed CoordinateTransformations ─ v0.5.0
 Installed BinDeps ─────────────────── v0.8.10
 Installed HTTP ────────────────────── v0.8.0
 Installed WoodburyMatrices ────────── v0.4.1
 Installed URIParser ───────────────── v0.4.0
 Installed OrderedCollections ──────── v1.1.0
 Installed Observables ─────────────── v0.2.3
 Installed JSExpr ──────────────────── v0.5.0
 Installed Lazy ────────────────────── v0.13.2
 Installed MacroTools ──────────────── v0.5.0
 Installed Hiccup ──────────────────── v0.2.2
 Installed WebSockets ──────────────── v1.5.2
 Installed ColorTypes ──────────────── v0.8.0
 Installed Colors ──────────────────── v0.9.5
 Installed AxisAlgorithms ──────────── v1.0.0
 Installed CSSUtil ─────────────────── v0.1.0
 Installed IterTools ───────────────── v1.1.1
 I

## Model definition

We'll just use the double pendulum model, loaded from a URDF:

In [2]:
srcdir = dirname(pathof(RigidBodyDynamics))
urdf = joinpath(srcdir, "..", "test", "urdf", "Acrobot.urdf")
mechanism = parse_urdf(urdf)

Spanning tree:
Vertex: world (root)
  Vertex: upper_link, Edge: shoulder
    Vertex: lower_link, Edge: elbow
No non-tree joints.

## Controller

Let's write a simple controller that just applies $10 \sin(t)$ at the elbow joint and adds some damping at the shoulder joint:

In [3]:
shoulder, elbow = joints(mechanism)
function simple_control!(torques::AbstractVector, t, state::MechanismState)
    torques[velocity_range(state, shoulder)] .= -1 .* velocity(state, shoulder)
    torques[velocity_range(state, elbow)] .= 10 * sin(t)
end;

## Simulation

Basic simulation can be done using the `simulate` function. We'll first create a `MechanismState` object, and set the initial joint configurations and velocities:

In [4]:
state = MechanismState(mechanism)
zero_velocity!(state)
set_configuration!(state, shoulder, 0.7)
set_configuration!(state, elbow, -0.8);

Now we can simply call `simulate`, which will return a tuple consisting of:
* simulation times (a `Vector` of numbers)
* joint configuration vectors (a `Vector` of `Vector`s)
* joint velocity vectors (a `Vector` of `Vector`s)

In [5]:
final_time = 10.
ts, qs, vs = simulate(state, final_time, simple_control!; Δt = 1e-3);

For access to lower-level functionality, such as different ways of storing or visualizing the data generated during the simulation, it is advised to simply pattern match the basic `simulate` function.

## Visualization

For visualization, we'll use [`MeshCatMechanisms`](https://github.com/JuliaRobotics/MeshCatMechanisms.jl), an external package based on RigidBodyDynamics.jl.

In [6]:
using MeshCatMechanisms

Create a `MechanismVisualizer` and open it in a new browser tab
(see [`MeshCat.jl`](https://github.com/rdeits/MeshCat.jl) for other options):

In [7]:
mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf))
OPEN_VISUALIZER = false
OPEN_VISUALIZER && open(mvis);

And animate:

In [8]:
MeshCatMechanisms.animate(mvis, ts, qs; realtimerate = 1.);

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*